In [2]:
import pandas as pd

In [17]:
def check_filter(filename: str, encoding: str):
  df = pd.read_csv(filename, encoding=encoding)
  total_length = len(df)
  for col in df.columns:
    has_null = df[col].isnull().any()
    if has_null:
      return f"{filename} has null value at {col} \n"
    if len(df[col]) != total_length:
      return f"{filename} has incorrect column length at {col} \n"
  df1 = df.drop_duplicates()
  if len(df1) != total_length:
    return f"{filename} has duplicate rows \n"
  return f"{filename} check filter success \n"

# columns are ['id', 'thread_number', 'timestamp', 'text', 'retweets', 'likes', 'replies']
def check_column_types(filename: str, encoding: str):
  df = pd.read_csv(filename, encoding=encoding)
  if (df['timestamp'].dtype != int) or (df['retweets'].dtype != int) or (df['likes'].dtype != int) or (df['replies'].dtype != int):
   return f"{filename} has incorrect type \n"
  has_negatives1 = (df['retweets'] < 0).any()
  has_negatives2 = (df['likes'] < 0).any()
  has_negatives3 = (df['replies'] < 0).any()
  if has_negatives1 or has_negatives2 or has_negatives3:
    return f"{filename} has negative engagement values \n"
  for thread in df['thread_number']:
    if type(thread) != str:
      return f"{filename} has incorrect type, thread_number not a string \n"
    if thread[0:6] != "Thread":
      return f"{filename} has incorrect name for thread_number \n"
  return f"{filename} check column type success \n"

In [27]:
check_filter("five_ten.csv", encoding="ISO-8859-1")
check_filter("ten_fifteen.csv", encoding="ISO-8859-1")
check_filter("fifteen_twenty.csv", encoding="ISO-8859-1")
check_filter("twenty_twentyfive.csv", encoding="ISO-8859-1")
check_filter("twentyfive_thirty.csv", encoding="ISO-8859-1")

check_column_types("five_ten.csv", encoding="ISO-8859-1")
check_column_types("ten_fifteen.csv", encoding="ISO-8859-1")
check_column_types("fifteen_twenty.csv", encoding="ISO-8859-1")
check_column_types("twenty_twentyfive.csv", encoding="ISO-8859-1")
check_column_types("twentyfive_thirty.csv", encoding="ISO-8859-1")

'twentyfive_thirty.csv check column type success \n'

In [ ]:
import chardet

# Detect encoding
with open('five_ten.csv', 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'ISO-8859-1', 'confidence': 0.7222415887282886, 'language': ''}
